## <center>Сложные соединения<center>

Интересующие нас данные хранятся в таблицах city, customer, driver, shipment, truck. Давайте внимательно их изучим. Ниже представлена ER-диаграмма (от англ. entity-relation, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

![](images/relations.jpg)

#### **Задание 1.1**

Укажите название города с максимальным весом единичной доставки.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
&ensp;&ensp;&ensp;&ensp;**NATURAL JOIN** sql.city c  
**GROUP BY**  
&ensp;&ensp;&ensp;&ensp;c.city_id,  
&ensp;&ensp;&ensp;&ensp;s.weight  
**HAVING** s.weight = **MAX**(s.weight)  
**ORDER BY** s.weight **DESC**  
**LIMIT** 1

Green Bay

Сколько различных производителей грузовиков перечислено в таблице truck?

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;**COUNT**(**DISTINCT** t.make)  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.truck t

3

Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту?

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;d.first_name  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
&ensp;&ensp;&ensp;&ensp;**JOIN** sql.driver d **ON** s.driver_id = d .driver_id  
**GROUP BY** d.driver_id, s.cust_id  
**ORDER BY COUNT**(s.ship_id) **DESC**  
**LIMIT** 1

Holger

Укажите дату первой по времени доставок в таблице shipment.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;s.ship_date  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
**ORDER BY** s.ship_date  
**LIMIT** 1  

08.01.2016

Укажите дату последней по времени доставок в таблице shipment.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;s.ship_date  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
**ORDER BY** s.ship_date **DESC**  
**LIMIT** 1

27.12.2017

Укажите имя клиента, получившего наибольшее количество доставок за 2017 год.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.cust_name  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
&ensp;&ensp;&ensp;&ensp;**JOIN** sql.customer c **ON** s.cust_id = c.cust_id  
**GROUP BY** c.cust_id  
**ORDER BY COUNT**(s.ship_id) **DESC**  
**LIMIT** 1

Autoware Inc

### <center>Принцип и условия работы UNION<center>

Мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм. Для этого напишем простой запрос:

**SELECT**  
&ensp;&ensp;&ensp;&ensp;book_name object_name,  
&ensp;&ensp;&ensp;&ensp;'книга' object_description  
**FROM** public.books  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;movie_title,   
&ensp;&ensp;&ensp;&ensp;'фильм'   
**FROM** sql.kinopoisk

Визуально произведённое нами действие можно представить следующим образом:

![](images/union_all.jpg)

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них **одинаковая структура**, а именно:

* одинаковый тип данных;
* одинаковое количество столбцов;
* одинаковый порядок столбцов согласно типу данных.

#### <center>Виды UNION<center>

Оператор присоединения существует в двух вариантах:

**UNION** выводит только уникальные записи;  
**UNION ALL** присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности. 

Важно! **UNION** оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с **DISTINCT**). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать **UNION ALL**.

#### <center>Ситнаксис<center>

Запрос строится таким образом:

**SELECT** n columns  
**FROM**   
&ensp;&ensp;&ensp;&ensp;table_1  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;n columns  
**FROM**  
&ensp;&ensp;&ensp;&ensp;table_2  
...  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;n columns  
**FROM**   
&ensp;&ensp;&ensp;&ensp;table_n    

Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.  
**Важно!** Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.


Сортировку любой части с использованием ORDER BY и LIMIT можно вести любой части запроса, обернув ее в круглые скобки.

(**SELECT** book_name object_name, 'книга' object_descritption  
**FROM** public.books  
**ORDER BY** 1  
**LIMIT** 1)  
**UNION ALL**  
(**SELECT** movie_title, 'фильм'   
**FROM** sql.kinopoisk  
**ORDER BY** 1  
**LIMIT** 1)

#### **Задание 2.1**

Напишите запрос, который создает уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков. Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck). Отсортируйте список по названию объекта, а затем — по типу.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;**DISTINCT** city_name object_name, 'city' object_type  
**FROM** sql.city  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;**DISTINCT** state, 'state'  
**FROM** sql.city  
**UNION ALL**  
**SELECT**
&ensp;&ensp;&ensp;&ensp;first_name, 'driver'  
**FROM** sql.driver  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;**DISTINCT** make, 'truck'  
**FROM** sql.truck  
**ORDER BY** object_name, object_type

#### **Задание 2.2**

Напишите запрос, который соберёт имена всех упомянутых городов и штатов из таблицы city. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

*Запрос:*

**SELECT** 
&ensp;&ensp;&ensp;&ensp;city_name object_name  
**FROM** sql.city   
**UNION ALL**    
**SELECT**  
    &ensp;&ensp;&ensp;&ensp;state  
**FROM** sql.city  
**ORDER BY** object_name

#### **Задание 2.3**

Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов. Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.  

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;city_name object_name  
**FROM** sql.city    
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;state  
**FROM** sql.city  
**ORDER BY** object_name

### <center>Ограничение типов данных<center>

**Важно!** Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_id::text  
**FROM** sql.city c  
**UNION ALL**    
**SELECT**  
&ensp;&ensp;&ensp;&ensp;cc.city_name  
**FROM** sql.city cc

#### **Задание 3.1**

Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник contact. Также добавьте столбец с именем водителя first_name и столбец contact_type с типом контакта (phone или zip в зависимости от типа). Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.  

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;zip_code::text contact,   
&ensp;&ensp;&ensp;&ensp;first_name first_name,   
&ensp;&ensp;&ensp;&ensp;'zip' contact_type  
**FROM** sql.driver  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;phone::text contact,   
&ensp;&ensp;&ensp;&ensp;first_name,  
&ensp;&ensp;&ensp;&ensp;'phone'  
**FROM** sql.driver  
**ORDER BY** contact, first_name  

#### <center>UNION ALL и промежуточнеы итоги<center>

UNION ALL часто используется для подведения **промежуточных итогов** и **выведения результатов агрегатных функций**.  
Кроме агрегатных функций, в запросах с UNION могут использоваться функции группировки и выборки.

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name,  
&ensp;&ensp;&ensp;&ensp;c.population  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.city c  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;'total',  
&ensp;&ensp;&ensp;&ensp;**SUM**(c.population)  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.city c  
**ORDER BY** 2 **DESC**

![](images/union_agr.jpg)

Выполнив запрос, вы заметите, что сначала выведен итоговый результат с суммой населения всех городов в таблице, а затем по каждому городу в отдельности.

#### **Задание 4.1**

Напишите запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. Необходимые столбцы: date_period, cnt_shipment. Не забывайте о единой типизации. Упорядочите по убыванию столбца date_period.  

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;s.ship_date::text date_period,  
&ensp;&ensp;&ensp;&ensp;**COUNT**(ship_id) cnt_shipment  
**FROM** sql.shipment s  
**GROUP BY** date_period  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;'total_shipments',  
&ensp;&ensp;&ensp;&ensp;**COUNT**(s.ship_id)  
**FROM** sql.shipment s  
**ORDER BY** date_period **DESC**

### <center>UNION и дополнительные условия<center>

**UNION** также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».  
Например, с помощью **UNION** можно отобразить, у кого из водителей заполнен столбец с номером телефона.

**SELECT**  
&ensp;&ensp;&ensp;&ensp;d.first_name,  
&ensp;&ensp;&ensp;&ensp;d.last_name,  
&ensp;&ensp;&ensp;&ensp;'телефон заполнен' phone_info  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.driver d 
**WHERE** d.phone **IS NOT NULL**  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;d.first_name,  
&ensp;&ensp;&ensp;&ensp;d.last_name,  
&ensp;&ensp;&ensp;&ensp;'телефон не заполнен' phone_info  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.driver d  
**WHERE** d.phone **IS NULL** 

#### **Задание 5.1**

Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:

* если в город была осуществлена доставка, то выводим 'доставка осуществлялась';
* если нет — выводим 'доставка не осуществлялась'.  

Столбцы к выводу: city_name, state, shipping_status. Отсортируйте в алфавитном порядке по городу, а затем — по штату.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name city_name,  
&ensp;&ensp;&ensp;&ensp;c.state state,  
&ensp;&ensp;&ensp;&ensp;'доставка осуществлялась' shipping_status  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.city c  
&ensp;&ensp;&ensp;&ensp;**LEFT JOIN** sql.shipment s **ON** c.city_id = s.city_id  
**GROUP BY** c.city_id  
**HAVING COUNT**(s.ship_id) > 0  
**UNION** 
**SELECT** 
&ensp;&ensp;&ensp;&ensp;c.city_name,  
&ensp;&ensp;&ensp;&ensp;c.state state,  
&ensp;&ensp;&ensp;&ensp;'доставка не осуществлялась'  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.city c  
&ensp;&ensp;&ensp;&ensp;**LEFT JOIN** sql.shipment s ON c.city_id = s.city_id    
**GROUP BY** c.city_id    
**HAVING COUNT**(s.ship_id) = 0  
**ORDER BY** city_name, state

#### **Задание 5.2**

Напишите запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки. Пусть первый столбец содержит название города, а второй формируется так:

* если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
* иначе — вывести количество доставок, увеличенное на пять.  

Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name city_name,  
&ensp;&ensp;&ensp;&ensp;COUNT(s.ship_id) shipping_fake  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.city c  
&ensp;&ensp;&ensp;&ensp;**LEFT JOIN** sql.shipment s **ON** c.city_id = s.city_id  
**GROUP BY** c.city_id  
**HAVING COUNT**(s.ship_id) > 10  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name,  
&ensp;&ensp;&ensp;&ensp;**COUNT**(s.ship_id) + 5  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.city c   
&ensp;&ensp;&ensp;&ensp;**LEFT JOIN** sql.shipment s **ON** c.city_id = s.city_id   
**GROUP BY** c.city_id     
**HAVING COUNT**(s.ship_id) **BETWEEN** 1 and 10  
**ORDER BY** shipping_fake **DESC**, city_name

### <center>UNION и ручная генерация<center>

**UNION** можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.  
Составим запрос, который позволит вывести первые три буквы алфавита и их порядковые номера.  

*Запрос:*

**SELECT**   
&ensp;&ensp;&ensp;&ensp;'a' letter,'1' ordinal_position  
**UNION**   
**SELECT**  
&ensp;&ensp;&ensp;&ensp;'b','2'  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;'c','3'  

Существуют сложные алгоритмы сравнения текстовых значений, но главный смысл сводится к одному: сравнение производится на основе таблицы unicode и позиции элемента в ней с учётом определённых условий.

#### **Задание 6.1**

Напишите запрос, который выберет наибольшее из значений:

* 1000000;  
* 541;  
* -500;  
* 100.  

Столбец с результатом назовите result.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;1000000 result   
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;541  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;-500  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;100  
**ORDER BY** result **DESC**  
**LIMIT** 1

1000000

#### **Задание 6.2**

Помним, что сортировка для числовых и строковых типов данных отличается. Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них:

* 1000000;
* 541;
* -500;
* 100.  

Столбец с ответом назовите mycol

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;1000000::text mycol  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;541::text  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;(-500)::text  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;100::text    
**ORDER BY** mycol **DESC**  
**LIMIT** 1

541

#### **Задание 6.3**

Построив запрос по аналогии с примером, найдите самое большое значение из перечисленных операторов:

* '+' ;  
* '-' ;  
* '=' ;  
* '/' .

Столбец с ответом назовите result.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;'+' result  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;'-'  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;'='  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;'/'  
**ORDER BY** result **DESC**  
**LIMIT** 1

'+'



### <center>Исключение повторяющихся данных<center>

Предположим, нам нужно узнать, в какие города осуществлялась доставка, **за исключением** тех, в которых проживают водители.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
&ensp;&ensp;&ensp;&ensp;**JOIN** sql.city c **ON** s.city_id = c.city_id  
**EXCEPT**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;cc.city_name  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.driver d  
&ensp;&ensp;&ensp;&ensp;**JOIN** sql.city cc **ON** d.city_id=cc.city_id  
**ORDER BY** 1

Как вы, должно быть, заметили, для решения этой задачи мы использовали оператор **EXCEPT**. Чтобы лучше понять данный тип присоединения, предлагаем ознакомиться с **диаграммой Венна** — математическим инструментом, представляющим возможные логические связи между соединёнными наборами данных.  

![](images/except.jpg)

Синтаксические правила для оператора EXCEPT такие же, как и для UNION:

одинаковый тип данных;
одинаковое количество столбцов;
одинаковый порядок столбцов согласно типу данных.
Синтаксис выглядит следующим образом:

**SELECT**  
&ensp;&ensp;&ensp;&ensp;n columns  
**FROM**  
&ensp;&ensp;&ensp;&ensp;table_1  
**EXCEPT**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;n columns  
**FROM**
&ensp;&ensp;&ensp;&ensp;table_2   

Мы уже знаем, как решить такую задачу с использованием LEFT JOIN. Вариант с EXCEPT будет полезен в тех случаях, когда у вас много столбцов и вам не хочется прописывать их равенство в условии для JOIN.

#### **Задание 7.1**

Выведите список zip-кодов, которые есть в таблице sql.driver, но отсутствуют в таблице sql.customer. Отсортируйте по возрастанию, столбец к выводу — zip. В поле ниже введите запрос, с помощью которого вы решили эту задачу.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;d.zip_code zip  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.driver d  
**EXCEPT**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.zip  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.customer c  
**ORDER BY** zip

### <center>Выборка общих данных из нескольких таблиц<center>

А что если нам надо вывести общие записи — те, что существуют в нескольких таблицах?  Предположим, нам надо вывести совпадающие по названию города и штаты.

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name object_name  
**FROM**            
&ensp;&ensp;&ensp;&ensp;sql.city c  
**INTERSECT**  
**SELECT**    
&ensp;&ensp;&ensp;&ensp;cc.state   
**FROM**            
&ensp;&ensp;&ensp;&ensp;sql.city cc   
**ORDER BY** 1  

Чтобы лучше понять, как работает этот оператор, вновь обратимся к **диаграмме Венна**: INTERSECT оставляет из результатов первого запроса все строки, которые совпали с результатом выполнения второго запроса.  

![](images/intersect.jpg)

Синтаксис запроса с оператором INTERSECT выглядит следующим образом:

**SELECT**  
&ensp;&ensp;&ensp;&ensp;n columns  
**FROM**  
&ensp;&ensp;&ensp;&ensp;table_1   
**INTERSECT**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;n columns  
**FROM**   
&ensp;&ensp;&ensp;&ensp;table_2  

#### **Задание 8.1**

Напишите запрос, который выведет список id городов, в которых есть и клиенты, и доставки, и водители.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_id id  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.city c  
**INTERSECT**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;cs.city_id 
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.customer cs  
**INTERSECT**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;s.city_id  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
**INTERSECT**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;d.city_id  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.driver d

#### **Задание 8.2**

Выведите zip-код, который есть как в таблице с клиентами, так и в таблице с водителями.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;cs.zip zip_code  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.customer cs  
**INTERSECT**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;d.zip_code  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.driver d

### <center>Итоговые задания<center>

#### **Задание 9.1**

Выведите города с максимальным и минимальным весом единичной доставки. Столбцы к выводу — city_name, weight.

*Запрос:*

(**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name city_name,  
&ensp;&ensp;&ensp;&ensp;**MAX**(s.weight) weight  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.city c   
&ensp;&ensp;&ensp;&ensp;**JOIN** sql.shipment s **ON** c.city_id = s.city_id  
**GROUP BY** c.city_id  
**ORDER BY** weight **DESC**  
**LIMIT** 1)  
**UNION**  
(**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name city_name,  
&ensp;&ensp;&ensp;&ensp;**MIN**(s.weight) weight  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.city c  
&ensp;&ensp;&ensp;&ensp;**JOIN** sql.shipment s **ON** c.city_id = s.city_id  
**GROUP BY** c.city_id  
**ORDER BY** weight  
**LIMIT** 1)


city_name &ensp;weight  
Green Bay&ensp;&ensp;85,581  
Saginaw&ensp;&ensp;&ensp;&ensp;477

#### **Задание 9.2**

Выведите идентификационные номера клиентов (cust_id), которые совпадают с идентификационными номерами доставок (ship_id). Столбец к выводу — mutual_id. Отсортируйте по возрастанию.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;cs.cust_id mutual_id  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.customer cs,  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
**WHERE** cs.cust_id = s.ship_id  
**ORDER BY** mutual_id

#### **Задание 9.3**

Создайте справочник, содержащий уникальные имена клиентов, которые являются производителями (cust_type='manufacturer'), и производителей грузовиков, а также описание объекта — 'КЛИЕНТ' или 'ГРУЗОВИК'. Столбцы к выводу — object_name, object_description. Отсортируйте по названию в алфавитном порядке.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;cs.cust_name object_name,  
&ensp;&ensp;&ensp;&ensp;'КЛИЕНТ' object_description  
**FROM** sql.customer cs  
**WHERE** cs.cust_type='manufacturer'  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;t.make,  
&ensp;&ensp;&ensp;&ensp;'ГРУЗОВИК'  
**FROM** sql.truck t  
**ORDER BY** object_name  